In [23]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import backend as K

from scipy.sparse import csr_matrix, vstack, hstack
from sklearn.metrics import roc_auc_score

In [24]:
train = pd.read_csv("train.csv").dropna()
test = pd.read_csv("test.csv").dropna()
y_train = np.array(train['Sentiment'])
y_test = np.array(test['Sentiment'])
X_train = train.drop(['Sentiment'], axis=1)
X_train = X_train.values
X_test = test.drop(['Sentiment'], axis=1).values
gen_features = ['word_count', 'unigram_count', 'unique_word_count', 'unique_unigram_count', 'stopword_count', 'mean_word_length', 'mean_unigram_length', 'char_count', 'punctuation_count', 'number_count']

In [17]:
import keras_tuner as kt

def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=16, max_value=256, step=16),
                    activation='relu', input_dim=X_train.shape[1],
                    kernel_regularizer=l2(hp.Float('l2', min_value=1e-4, max_value=1e-2, sampling='LOG'))))
    model.add(Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(1, activation='tanh'))

    adam = Adam(learning_rate=hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG'))
    model.compile(optimizer=adam, loss='mean_squared_error', metrics=['accuracy'])

    return model

# Create a tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=50,
    hyperband_iterations=2)

# Perform hyperparameter tuning
tuner.search(X_train, y_train, epochs=50, validation_split=0.1, callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]


Trial 20 Complete [00h 00m 01s]
val_accuracy: 0.0

Best val_accuracy So Far: 0.0
Total elapsed time: 00h 00m 12s

Search: Running Trial #21

Value             |Best Value So Far |Hyperparameter
224               |32                |units
0.0089263         |0.00010518        |l2
0.1               |0.4               |dropout
0.0010326         |0.00011517        |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
3                 |3                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
  1/166 [..............................] - ETA: 22s - loss: 3.6147 - accuracy: 0.5000

KeyboardInterrupt: 

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers.legacy import Adam

model = Sequential()
model.add(Dense(16, activation='relu', input_dim=X_train.shape[1], kernel_regularizer=l2(0.001)))

model.add(Dense(8, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dense(1, activation='tanh'))  # Output layer


def custom_binary_crossentropy(y_true, y_pred):
    # Transform y_true to 0 and 1.
    y_true_transformed = K.cast((y_true + 1) / 2, dtype='float64')
    y_pred_transformed = K.cast((y_pred + 1) / 2, dtype='float64')

    # Compute binary crossentropy
    bce = K.binary_crossentropy(y_true_transformed, y_pred_transformed, from_logits=False)
    
    return K.mean(bce)

adam = Adam(learning_rate=0.0002)
def custom_accuracy(y_true, y_pred):
    # Since y_pred comes from tanh, values will be between -1 and 1.
    # We convert these to 1 and -1 for comparison.
    y_pred_labels = K.cast(K.greater(y_pred, 0), K.floatx()) * 2 - 1
    # Compare predicted labels to true labels
    correct_predictions = K.equal(y_true, y_pred_labels)
    # Calculate accuracy
    return K.mean(correct_predictions, axis=-1)

model.compile(optimizer=adam, loss=custom_binary_crossentropy, metrics=[custom_accuracy])
# Fit the model - assuming your data is ready and appropriately preprocessed
model.fit(X_train, y_train, epochs=20, batch_size=4, validation_split=0.1)

Epoch 1/20
1326/1326 [==============================] - 1s 599us/step - loss: 1.0256 - custom_accuracy: 0.5324 - val_loss: 0.0929 - val_custom_accuracy: 1.0000
Epoch 2/20
1326/1326 [==============================] - 1s 485us/step - loss: 0.7630 - custom_accuracy: 0.5715 - val_loss: 0.1819 - val_custom_accuracy: 1.0000
Epoch 3/20
1326/1326 [==============================] - 1s 490us/step - loss: 0.8010 - custom_accuracy: 0.5997 - val_loss: 1.0241 - val_custom_accuracy: 0.0915
Epoch 4/20
1326/1326 [==============================] - 1s 486us/step - loss: 0.7319 - custom_accuracy: 0.6054 - val_loss: 1.0606 - val_custom_accuracy: 0.0797
Epoch 5/20
1326/1326 [==============================] - 1s 485us/step - loss: 0.7012 - custom_accuracy: 0.6214 - val_loss: 1.8498 - val_custom_accuracy: 0.0017
Epoch 6/20
1326/1326 [==============================] - 1s 508us/step - loss: 0.7369 - custom_accuracy: 0.6192 - val_loss: 2.7703 - val_custom_accuracy: 0.0000e+00
Epoch 7/20
1326/1326 [==============

In [68]:

# You can now access the best hyperparameters as follows:
best_units = best_hps.get('units')
best_dropout = best_hps.get('dropout')
best_l2 = best_hps.get('l2')
best_learning_rate = best_hps.get('learning_rate')

# Now you can use these best hyperparameters to build the best model
best_model = tuner.hypermodel.build(best_hps)

# Summary of the best model
best_model.summary()

# You can now train the best model with the optimal hyperparameters
history = best_model.fit(X_train, y_train)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 208)               256880    
                                                                 
 dropout_2 (Dropout)         (None, 208)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 209       
                                                                 
Total params: 257089 (1004.25 KB)
Trainable params: 257089 (1004.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
185/185 [==============================] - 0s 1ms/step - loss: 3.8837 - accuracy: 0.2735


In [32]:
y_pred_test = model.predict(X_test)
roc_auc_test = roc_auc_score(y_test, y_pred_test)
print(f'AUC: {roc_auc_test}')

37/37 [==============================] - 0s 383us/step
AUC: 0.7531203934367134
